## Robot@Home 2 - Concatenating images from multiple cameras `v0.1`

`R@H2 notebook series`   

<a href="https://colab.research.google.com/github/goyoambrosio/RobotAtHome2/blob/master/notebooks/50-RGBD-observations.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>



### R@H2 functions introduced in this notebook:



-   `process_image`
-   `concat_images`


### Getting started



Install Robot@Home2 Toolbox using the Python package manager



In [ ]:
!pip install robotathome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.6 MB/s eta 0:00:00


Now, let's mount Google Drive (more info in [this notebook](https://colab.research.google.com/github/goyoambrosio/RobotAtHome2/blob/master/notebooks/05-Google-colab-drive.ipynb)) and instantiate the RobotAtHome class.

In [ ]:
from google.colab import drive
from robotathome import RobotAtHome
from robotathome import logger, log, set_log_level

# Let's mount Google Drive
drive.mount('/content/drive')

# Then copy the provided configutarion file to the current directory
!cp /content/drive/MyDrive/R@H2-2.0.3/notebooks/.rh .

# And create an instance of the RobotAtHome class
try: 
      rh = RobotAtHome()
except:
      logger.error("Something was wrong")


Mounted at /content/drive


2023-04-17 07:09:23.900 | SUCCESS  | robotathome.core.reader:__open_dataset:141 - Connection is established: rh.db


### Iterating R@H2 over RGB-D images coming from multiple cameras and concatenating them




In this example, we are going to iterate over RGBD images coming from the four cameras.
As the previous notebook, we are going to iterate over labeled RGBD images from **master room** of the **alma** home.

To begin with, as in the previous examples, we are going to work with the set of labeled images where we will select the observations according to the selection criteria.

In [ ]:
from robotathome import filter_sensor_observations
from robotathome import composed_RGBD_images
from robotathome import concat_images
from robotathome import process_image

# cv2 causes some trouble to Colab so they provide a patch
# Usually you'll write: from cv2 import imshow
from google.colab.patches import cv2_imshow

log.set_log_level('INFO')  # SUCCESS is the default

# Fill the variables that constitutes selection filter
home_session_name = 'alma-s1'
home_subsession = 0
room_name = 'alma_masterroom1'
sensor_list = ['RGBD_3', 'RGBD_4', 'RGBD_1', 'RGBD_2'] # Left to right order

# Get the labeled RGB-D observations dataframe
df = rh.get_sensor_observations('lblrgbd')

# Filter the dataframe and get a dictionary with a dataframe per sensor
df_dict = filter_sensor_observations(rh, df,
                                     home_session_name,
                                     home_subsession,
                                     room_name,
                                     sensor_list)

logger.info(f"Labeled RGBD set: {len(df)} observations")
for sensor_name in sensor_list:
    logger.info(f"No. of RGBD observations int the filtered subset for the sensor {sensor_name}: {len(df_dict[sensor_name])} observations")

2023-04-17 07:09:25.124 | INFO     | __main__:<cell line: 28>:28 - Labeled RGBD set: 32937 observations
2023-04-17 07:09:25.125 | INFO     | __main__:<cell line: 29>:30 - No. of RGBD observations int the filtered subset for the sensor RGBD_3: 299 observations
2023-04-17 07:09:25.128 | INFO     | __main__:<cell line: 29>:30 - No. of RGBD observations int the filtered subset for the sensor RGBD_4: 299 observations
2023-04-17 07:09:25.130 | INFO     | __main__:<cell line: 29>:30 - No. of RGBD observations int the filtered subset for the sensor RGBD_1: 299 observations
2023-04-17 07:09:25.131 | INFO     | __main__:<cell line: 29>:30 - No. of RGBD observations int the filtered subset for the sensor RGBD_2: 299 observations


In Google Colab, reading files from the mounted drive takes time, so for this example we will select only a few images.

In [ ]:
# For this example, we will only select a few images
df_RGBD_N_only_some_frames = {}
for sensor_name in sensor_list:
    df_RGBD_N_only_some_frames[sensor_name] = df_dict[sensor_name][30:35]

As we already know, the iteration is carried out through a main loop. The novelty in this notebook comes from the `process_image` function that takes two arguments: a function and a dictionary of images (an image per camera sensor). `process_image` applies `f` to the dictionary of images.

In this example, we want to join the images coming from the selected cameras (remember `sensor_list`). That is the point where the new `concat_images` function takes place. That function receives a dictionary of images (the keys are the names of the sensors) and returns an image made up of the others.

In [ ]:
# Iterate over the dictionary of dataframes, i.e. frame by frame
for (RGB_image_dict, D_image_dict) in composed_RGBD_images(rh, df_RGBD_N_only_some_frames):
    def f(img_dict):
        # return my_function(img_dict, par1,... parn)
        return concat_images(img_dict)

    # Apply f to img_dict
    resulting_img = process_image(f, RGB_image_dict)
    # Show the new output image
    cv2_imshow(resulting_img)

Output hidden; open in https://colab.research.google.com to view.